# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [16]:
import pandas as pd
import numpy as np
from scipy import stats
import math
# To have multiple outputs in one cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [11]:
# number of callbacks for White/black-sounding names
sum(data[data.race=='w'].call)
sum(data[data.race=='b'].call)

235.0

157.0

In [9]:
pd.set_option('display.max_columns', 50)
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,occupbroad,workinschool,email,computerskills,specialskills,firstname,sex,race,h,l,call,city,kind,adid,fracblack,...,fracwhite_empzip,lmedhhinc_empzip,fracdropout_empzip,fraccolp_empzip,linc_empzip,manager,supervisor,secretary,offsupport,salesrep,retailsales,req,expreq,comreq,educreq,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,1,0,0,1,0,Allison,f,w,0.0,1.0,0.0,c,a,384.0,0.989360,...,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,6,1,1,1,0,Kristen,f,w,1.0,0.0,0.0,c,a,384.0,0.080736,...,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,1,1,0,1,0,Lakisha,f,b,0.0,1.0,0.0,c,a,384.0,0.104301,...,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,5,0,1,1,1,Latonya,f,b,1.0,0.0,0.0,c,a,384.0,0.336165,...,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,5,1,1,1,0,Carrie,f,w,1.0,0.0,0.0,c,a,385.0,0.397595,...,0.727046,10.619399,0.070493,0.369903,10.007352,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

# 1. What test is appropriate for this problem? Does CLT apply?

###### The goal is to establish whether race has a significant impact on the rate of callbacks for resumes. testing the difference between the percentage called back for each race will follow a normal distribution in which CLT can be applied which states that the difference between the two proportions is approximately a normal distribution. Two sample t-test is appropriate to use in comparing these two percentages.

###### In summary:
#### Two-sample t-test is appropiate because we are trying to determine if two population means are equal.
#### CLT applies because our sample size is large (n = 4870) and observations are independent.

# 2. What are the null and alternate hypotheses?

#### H0: the mean difference between callback rate for black & white-sounding names is zero                               
#### H1: there is a difference between callback rate for black & white-sounding names

In [12]:
w = data[data.race=='w']
b = data[data.race=='b']

# 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [14]:
# calculate statistics for binomial distributions
n1 = len(data[data['race'] == 'b']) # black sample size
n2 = len(data[data['race'] == 'w']) # white sample size
p1 = sum(data[data['race'] == 'b'].call)/n1 # black callback rate
p2 = sum(data[data['race'] == 'w'].call)/n2 # white callback rate

In [17]:
# conduct two-sample t-test
p_hat = (n1*p1+n2*p2)/(n1+n2) # joint callback
std_hat = math.sqrt(p_hat*(1-p_hat)*((1/n1)+(1/n2))) # joint standard deviation

test_stat = (p1-p2)/std_hat
p_val = stats.t.sf(abs(test_stat), df=len(data)-1)*2

print('test statistic:',test_stat)
print('p value:',p_val)

test statistic: -4.108412152434346
p value: 4.0493178875903686e-05


In [19]:
# calculate margins of error for the two distributions
stderr1 = math.sqrt((p1*(1-p1))/n1) # black standard error
stderr2 = math.sqrt((p2*(1-p2))/n2) # white standard error
stderr_diff = math.sqrt(math.pow(stderr1,2) + math.pow(stderr2,2)) # standard error for difference

# calculate confidence interval for difference in the distributions
conf_int_lower = abs(p1-p2)-1.96*stderr_diff
conf_int_upper = abs(p1-p2)+1.96*stderr_diff
conf_int = [conf_int_lower,conf_int_upper]

print('confidence interval for the difference of black & white callback rate:')
print(conf_int)

confidence interval for the difference of black & white callback rate:
[0.016777447859559147, 0.047288260559332024]


#### While the p-value is < 0.05, the Ho will be rejected and H1 will be accepted. There is significance difference between black and white name sounding resumes looking at percentage callbacks.

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

# 4. Write a story describing the statistical significance in the context or the original problem.

#### With a p-value of less than 0.05 and confidence interval that does not contain 0, we can conclude that there is a statistically significant difference between callback rates for black and white-sounding names.

# 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

#### No, the conclusion above does not imply that race is the most important factor in callback success, because it only examines race. There are other variables like education and years of experience that we may need to examine and analyze to reach into a solid conclusion.